# Step 1: Defining the Study Area

In [ ]:
import os
import yaml
from pathlib import Path
import geopandas as gpd
import rasterio

# -----------------------------------------------------------------------------
# Load config.yml
# -----------------------------------------------------------------------------

# Get project root (adjust based on your folder depth)
current_dir = Path(os.getcwd())
project_root = current_dir.parent.parent  # Navigate up from "Scripts/Phase1_Data_Preprocessing"

with open(project_root / "config.yml", "r") as f:
    config = yaml.safe_load(f)

# -----------------------------------------------------------------------------
# Construct paths
# -----------------------------------------------------------------------------

# Raw data paths
raw_data_dir = project_root / config["paths"]["raw_data"]
soil_raw_dir = raw_data_dir / "GIS/Soil"  # Matches your hardcoded path structure
morocco_path = raw_data_dir / config["paths"]["morocco_boundary"]
tadla_plain_path = raw_data_dir / config["paths"]["tadla_plain_raw"]
tadla_plain_boundary_path = raw_data_dir / config["paths"]["tadla_plain_boundary_raw"]
soil_raw_path = raw_data_dir / config["paths"]["soil_raw"]

# Processed data paths
processed_data_dir = project_root / config["paths"]["processed_data"]
output_dir = processed_data_dir / "GIS/Study_Area_Boundary"
output_path = output_dir / "Tadla_plain_common.shp"
tadla_common_path = processed_data_dir / config["paths"]["tadla_boundary_processed"]
soil_processed_path = processed_data_dir / config["paths"]["soil_processed"]

In [ ]:
# Load Morocco boundary
morocco = gpd.read_file(morocco_path)

# Check the first few rows to see province names
morocco.head()

In [ ]:
morocco.plot()

In [ ]:
# Load Tadla Plain shapefile
tadla_plain_polygon = gpd.read_file(tadla_plain_path)

# Check the data
print(tadla_plain_polygon)  # Show first few rows


In [ ]:
tadla_plain_polygon.plot()  # Plot the geometry

In [ ]:
print(f"Study area size: {tadla_plain_polygon.geometry.area} m²") 

In [ ]:
# Reproject to Merchich (EPSG:26191)
tadla_merchiche = tadla_plain_polygon.to_crs(epsg=26191)

# Calculate area
area_m2 = tadla_merchiche.geometry.area
print(f"Study area size: {area_m2[0]:.2f} m²")  
# Example output: "Study area size: 1300000000.00 m²"

area_ha = area_m2 / 10000
print(f"Study area size: {area_ha[0]:.2f} hectares")  
# Example output: "Study area size: 130000.00 hectares"


In [ ]:
tadla_merchiche.plot()

In [ ]:
# Load the cleaned boundary shapefile
Tadla_plain_boundary = gpd.read_file(tadla_plain_boundary_path)
# Check the current CRS
print(Tadla_plain_boundary.crs)

In [ ]:
# Convert to Merchich CRS if needed
if Tadla_plain_boundary.crs != "EPSG:26191":
    Tadla_plain_boundary = Tadla_plain_boundary.to_crs(epsg=26191)


In [ ]:
Tadla_plain_boundary.plot()

In [ ]:
# Assume these are already loaded and in the same CRS (EPSG:26191)
# tadla_merchiche: full administrative boundary (Merchich)
# tadla_plain_polygone: digitized Tadla plain (which may be slightly off)

# Compute the common (intersecting) area between the two layers
tadla_plain = gpd.overlay(Tadla_plain_boundary, tadla_merchiche, how='intersection')

# Save the resulting common area shapefile for further analysis
tadla_plain.to_file(output_path)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Plot layers with explicit labels
fig, ax = plt.subplots(figsize=(8, 8))
tadla_merchiche.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=2)
Tadla_plain_boundary.plot(ax=ax, facecolor="blue", alpha=0.5, edgecolor="blue")
tadla_plain.plot(ax=ax, facecolor="green", alpha=0.5, edgecolor="black")

# Create custom legend
legend_labels = {
    "Full Admin Boundary": "red",
    "Digitized Tadla Plain": "blue",
    "Common Area": "green"
}
patches = [Patch(color=color, label=label) for label, color in legend_labels.items()]
plt.legend(handles=patches)

plt.title("Common Area between Tadla Plain and Full Admin Boundary")
plt.show()

In [ ]:
tadla_plain = tadla_plain.to_crs(epsg=26191)  # Ensure projection
tadla_merchiche = tadla_merchiche.to_crs(epsg=26191)

area_plain_m2 = tadla_plain.geometry.area.sum()
area_full_m2 = tadla_merchiche.geometry.area.sum()

print(f"Tadla Plain area: {area_plain_m2:.2f} m²")
print(f"Full Admin Boundary area: {area_full_m2:.2f} m²")


In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject

def reproject_raster(input_path, output_path, target_crs):
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        metadata = src.meta.copy()
        metadata.update({
            "crs": target_crs,
            "transform": transform,
            "width": width,
            "height": height
        })

        with rasterio.open(output_path, "w", **metadata) as dest:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dest, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs
            )

# Step 2: Downloading Soil Data (SoilGrids)

In [ ]:
import geopandas as gpd

# Load Tadla boundary (EPSG:26191)
tadla = gpd.read_file(tadla_common_path)
tadla = tadla.to_crs("EPSG:26191")

# Get bounding box in Merchich coordinates
minx, miny, maxx, maxy = tadla.total_bounds
print(f"X: {minx}, {maxx}")  # Easting bounds
print(f"Y: {miny}, {maxy}")  # Northing bounds

### 1. Defining Parameters

In [ ]:
# Bounding box of Tadla Plain in EPSG:26191 (from your URL)
minx, maxx = 339200, 459750  # X (Easting)
miny, maxy = 164400, 241200  # Y (Northing)

# Soil layers and their COVERAGEIDs (adjust if needed)
layers = {
    "clay": "clay_0-5cm_mean",
    "silt": "silt_0-5cm_mean",
    "sand": "sand_0-5cm_mean",
    "ocd": "ocd_0-5cm_mean",    # Organic carbon density
    "wcs": "wcs_0-5cm_mean"     # Water content at saturation
}

### 2. Python Script to Download All Layers

In [ ]:
import requests
import os

os.makedirs(soil_raw_dir, exist_ok=True)

for param, coverage_id in layers.items():
    url = (
        f"https://maps.isric.org/mapserv?map=/map/{param}.map&"
        f"SERVICE=WCS&"
        f"VERSION=2.0.1&"
        f"REQUEST=GetCoverage&"
        f"COVERAGEID={coverage_id}&"
        f"FORMAT=GEOTIFF_INT16&"  # Or GEOTIFF_FLOAT32 for raw values
        f"SUBSET=X({minx},{maxx})&"
        f"SUBSET=Y({miny},{maxy})&"
        f"SUBSETTINGCRS=http://www.opengis.net/def/crs/EPSG/0/26191&"
        f"OUTPUTCRS=http://www.opengis.net/def/crs/EPSG/0/26191"
    )
    
    # Download and save
    response = requests.get(url)
    if response.status_code == 200:
        output_path = os.path.join(soil_raw_dir, f"tadla_{param}.tif")
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {param} to {output_path}")
    else:
        print(f"Failed to download {param}: HTTP {response.status_code}")

### 3. Post-Processing

1. Unit Conversion:

    SoilGrids stores integer values as actual value × 10. 
    
    For example:
        A pixel value of 150 = 15% clay.

In [ ]:
import rasterio
import numpy as np

# Process soil data

    # = src.profile
   

with rasterio.open(soil_raw_path) as src:
    clay = src.read(1)
    clay = clay.astype(np.float32) / 10  # Convert to %
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32)

    with rasterio.open(soil_processed_path, "w", **profile) as dst:
        dst.write(src.read())


2. Validate CRS Alignment

    Confirm all downloaded rasters are in EPSG:26191

In [ ]:
import rasterio

with rasterio.open(soil_raw_path) as src:
    print(src.crs)  # Should print "EPSG:26191"